In [3]:
import pandas as pd
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

from konlpy.tag import Okt

from collections import Counter
from wordcloud import WordCloud
import networkx as nx
from PIL import Image

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity


# 1. 데이터 불러오기

In [ ]:
df = pd.read_csv(
    "",
    index_col=0
)

# 2. 데이터 전처리

## 1) 데이터 확인

## 2) 데이터 형변환

## 3) 데이터 결측치 확인 및 처리

## 4) 데이터 이상치 확인 및 처리

## 5) 파생변수 생성하기

# 3. 데이터 탐색

## 1) 변수별 특징 파악

## 2) 그룹별 특징 파악

## 3) 변수간 관계 파악

#### chatGPT 협업 방식

In [ ]:
"""
[내 정보]
나는 파이썬을 처음 다뤄봐.
내 목표는 데이터 분석을 하는데 내가 관련 정보를 찾아서 직접 코드를 작성하는거야. 

[역할 및 목표]
너는 파이썬을 잘 활용할 줄 아는 데이터 분석가야. 
내가 데이터 정보와 결과를 주면 데이터 분석 과정에 맞게 
1. 내 수준에 맞게 결과를 해석하고, 
2. 새롭게 분석할 데이터 아이디어와 의도를 설명하고,
3. 내가 스스로 그 문제를 해결할 수 있게 파이썬 힌트를 주면 돼. 

내가 데이터 분석을 이해하고 수행할 수 있게 도와주는게 네 목표야.

[주의사항]
내가 물어보기 전까지는 전체 코드를 제공하지마. 

[데이터 분석 과정]
1. 데이터 불러오기
2. 데이터 전처리
    1) 데이터 정보 확인하기 
    2) 데이터 형 변환결측치 확인 및 처리
    3) 결측치 확인 및 처리
    4) 이상치 확인 및 처리
    5) 파생변수 생성
3. 데이터 탐색 
    1) 변수별 특징 파악하기
    2) 그룹별 특징 파악하기
    3) 변수 간 관계 파악하기
"""

# 4. TF-IDF 벡터라이져

In [7]:
docs = [
    "기분이 좋다",
    "오늘 기분이 나쁘다",
    "날씨가 좋다"
]

# 1) TF-IDF (docs x terms)
vec = TfidfVectorizer()  # 한국어면 tokenizer를 형태소로 바꾸는 게 보통 더 좋음
X = vec.fit_transform(docs)
terms = vec.get_feature_names_out()


In [41]:
# 내용 ( 순서 정보는 없어짐 )
print(X.toarray())
[terms[np.where(x>0, True, False)] for x in X.toarray()]

[[0.70710678 0.         0.         0.         0.70710678]
 [0.4736296  0.62276601 0.         0.62276601 0.        ]
 [0.         0.         0.79596054 0.         0.60534851]]


[array(['기분이', '좋다'], dtype=object),
 array(['기분이', '나쁘다', '오늘'], dtype=object),
 array(['날씨가', '좋다'], dtype=object)]

np.float64(0.7959605415681652)

# 5. CONCOR 분석 시각화 파일 만들기

In [ ]:
# 2-A) term x term cosine similarity
M_cos = cosine_similarity(X.T)  # (terms x terms)

# 2-B) term x term Pearson correlation (optional)
M_corr = np.corrcoef(X.toarray().T)  # (terms x terms)

# 원하는 걸 선택
M = M_cos  # or M_corr

# (선택) 대각선 처리: UCINET에서 보통 self-tie는 0 또는 1로 둠 (둘 다 가능)
np.fill_diagonal(M, 0.0)

dfM = pd.DataFrame(M, index=terms, columns=terms)

# 3) UCINET DL(fullmatrix)로 저장
def save_ucinet_dl_fullmatrix(df, path):
    labels = ", ".join(df.index.tolist())
    n = df.shape[0]
    with open(path, "w", encoding="CP949") as f:
        f.write(f"DL n={n}\n")
        f.write("format=fullmatrix\n")
        f.write("labels:\n")
        f.write(labels + "\n")
        f.write("data:\n")
        for row in df.values:
            f.write(" ".join(f"{v:.6f}" for v in row) + "\n")

save_ucinet_dl_fullmatrix(dfM, "word_similarity.dl")
print("Saved: word_similarity.dl")